In [ ]:

import pandas as pd
import requests

# 1. Sales CRM Users
users = requests.get("https://dummyjson.com/users").json()["users"]
for user in users:
    if "company" in user and isinstance(user["company"], dict):
        user["company_name"] = user["company"].get("name", "")
    else:
        user["company_name"] = ""
users_df = pd.DataFrame(users)
users_df["fullName"] = users_df["firstName"] + " " + users_df["lastName"]
users_cleaned = users_df[["id", "fullName", "email", "company_name"]]

# 2. Customer Support Tickets
tickets = requests.get("https://dummyjson.com/todos").json()["todos"]
tickets_df = pd.DataFrame(tickets)
tickets_cleaned = tickets_df[["id", "todo", "completed", "userId"]]

# 3. Inventory Products
products = requests.get("https://fakestoreapi.com/products").json()
products_df = pd.DataFrame(products)
if 'rating' in products_df.columns:
    # Some ratings are nested dicts
    products_df["rate"] = products_df["rating"].apply(lambda x: x["rate"] if isinstance(x, dict) and "rate" in x else None)
    products_df["count"] = products_df["rating"].apply(lambda x: x["count"] if isinstance(x, dict) and "count" in x else None)
products_cleaned = products_df[["id", "title", "price", "category", "rate", "count"]]

# 4. Payments (DummyJSON carts)
cards = requests.get("https://dummyjson.com/carts").json()["carts"]
cards_df = pd.DataFrame(cards)
cards_cleaned = cards_df[["id", "userId", "total", "discountedTotal", "totalProducts"]]

# 5. Orders (FakeStore carts)
orders = requests.get("https://fakestoreapi.com/carts").json()
orders_df = pd.DataFrame(orders)
orders_cleaned = orders_df[["id", "userId", "date"]]

# 6. Product Reviews
reviews = requests.get("https://dummyjson.com/comments").json()["comments"]
reviews_df = pd.DataFrame(reviews)
reviews_cleaned = reviews_df[["id", "body", "postId", "user"]]
# Flatten the nested 'user' column
reviews_cleaned["user_fullname"] = reviews_cleaned["user"].apply(lambda x: x["username"] if isinstance(x, dict) else None)
reviews_cleaned = reviews_cleaned.drop(columns=["user"])

# ✅ Display cleaned data
print("🧑‍💼 Sales CRM Users")
display(users_cleaned.head())

print("\n🎫 Customer Support Tickets")
display(tickets_cleaned.head())

print("\n📦 Inventory Products")
display(products_cleaned.head())

print("\n💳 Payments Data")
display(cards_cleaned.head())

print("\n🛒 Orders Data")
display(orders_cleaned.head())

print("\n📝 Product Reviews")
display(reviews_cleaned.head())